In [ ]:
import sys
import functools
from collections import deque

In [ ]:
fn = 'day18.txt'

In [ ]:
def load_grid(fn):
    grid = {}
    
    with open(fn) as fh:
        for row, line in enumerate(fh):
            for col, c in enumerate(line.strip()):
                grid[complex(col, row)] = c
                if c == '@':
                    entrance = complex(col, row)

    return grid, entrance

In [ ]:
def draw_grid(grid):
    max_x = max(int(p.real) for p in grid)
    max_y = max(int(p.imag) for p in grid)
    
    for row in range(max_y + 1):
        for col in range(max_x + 1):
            sys.stdout.write(grid[complex(col, row)])
        sys.stdout.write('\n')

In [ ]:
def distances_from(start, grid, keys_available):
    distances = {start: 0}
    to_check = deque()

    to_check.append(start)

    while len(to_check) > 0:
        position = to_check.popleft()
        for direction in [1, -1, 1j, -1j]:
            next_position = position + direction
            next_distance = distances[position] + 1
            next_char = grid[next_position]
            
            if next_char != '#' and ((not next_char.isupper()) or next_char in keys_available) and next_position not in distances:
                distances[next_position] = next_distance
                to_check.append(next_position)
    
    return distances

In [ ]:
def get_options(current_position, grid, keys_available):
    accessible = distances_from(current_position, grid, keys_available)
    options = {}
    for position in accessible:
        if position != current_position and grid[position].islower() and grid[position].upper() not in keys_available:
            options[grid[position]] = (position, accessible[position])
    return options

In [ ]:
@functools.lru_cache(maxsize=None)
def shortest_path_from(current_position, keys_available):

    options = get_options(current_position, grid, keys_available)

    if len(options) == 0:
        return 0
    
    possibilities = []
    
    for next_key, (next_position, distance_to) in options.items():
        new_keys = set(keys_available)
        new_keys.add(next_key.upper())
        new_keys = tuple(sorted(new_keys))
        remaining_distance_from = shortest_path_from(next_position, new_keys)
        possibilities.append(distance_to + remaining_distance_from)
        
    return min(possibilities)

In [ ]:
fn = 'day18.txt'

In [ ]:
%%time

grid, entrance = load_grid(fn)

print(shortest_path_from(entrance, tuple()))

In [ ]:
def load_grid_part2(fn):
    grid = {}
    entrances = []
    with open(fn) as fh:
        for row, line in enumerate(fh):
            for col, c in enumerate(line.strip()):
                grid[complex(col, row)] = c
                if c == '@':
                    entrances.append(complex(col, row))

    return grid, tuple(entrances)

In [ ]:
def partition_grid(fn):
    grid, entrance = load_grid(fn)
    entrances = []
    grid[entrance] = '#'
    for d in [1, -1, 1j, -1j]:
        grid[entrance + d] = '#'
    for d in [(1 + 1j), (1 - 1j), (-1 + 1j), (-1 - 1j)]:
        grid[entrance + d] = '@'
        entrances.append(entrance + d)
    return grid, tuple(entrances)

In [ ]:
def get_options_multiple(current_positions, grid, keys_available):
    options = {}
    
    for robot, current_position in enumerate(current_positions):
        accessible = distances_from(current_position, grid, keys_available)

        for position in accessible:
            if position != current_position and grid[position].islower() and grid[position].upper() not in keys_available:
                options[grid[position]] = (robot, position, accessible[position])
    
    return options

In [ ]:
@functools.lru_cache(maxsize=None)
def shortest_path_from_multiple(current_positions, keys_available):
    options = get_options_multiple(current_positions, grid, keys_available)

    if len(options) == 0:
        return 0
    
    possibilities = []
    
    for next_key, (robot, next_position, distance_to) in options.items():
        new_keys = set(keys_available)
        new_keys.add(next_key.upper())
        new_keys = tuple(sorted(new_keys))
        
        next_positions = list(current_positions)
        next_positions[robot] = next_position
        next_positions = tuple(next_positions)
        
        remaining_distance_from = shortest_path_from_multiple(next_positions, new_keys)
        possibilities.append(distance_to + remaining_distance_from)

    return min(possibilities)

In [ ]:
grid, entrances = load_grid_part2('day18_part2_test2.txt')

print(shortest_path_from_multiple(entrances, tuple()))

In [ ]:
%%time

grid, entrances = partition_grid('day18.txt')

shortest_path_from_multiple.cache_clear()
print(shortest_path_from_multiple(entrances, ()))